# Lab 1/2 EMG Processing

This lab will go into the basics of real-time classification using EMG signals. 

In [1]:
import pandas as pd
import numpy as np
import scipy
import scipy.signal
import matplotlib.pyplot as plt

In [2]:
''' If you have pyqt installed, this command will pop out interactive windows for graphs'''
%matplotlib qt

# Myo Data Processing
The following cells are for EMG data processing from the sample file

In [3]:
# Lets read in our data
import os
directory = 'Data/'
path = 'p18_emg.csv'
myo_df = pd.read_csv(directory + path)
myo_df.columns = myo_df.columns.str.replace(' ', '')
myo_df = myo_df.groupby('Arm').get_group('left') # This only needs to be done if you have two Myos running at the same time
display(myo_df)

,DeviceID,Warm?,Sync,Arm,Timestamp,Orientation_W,Orientation_X,Orientation_Y,Orientation_Z,Acc_X,...,EMG_4,EMG_5,EMG_6,EMG_7,EMG_8,Locked,RSSI,Roll,Pitch,Yaw
0,2291479052128,warm,True,left,2019-02-14 14:46:53 744402,-0.514465,-0.485779,0.287659,-0.645508,1.326660,...,-2,1,-1,1,-1,False,0,0.398451,-1.233521,1.567341
1,2291479052128,warm,True,left,2019-02-14 14:46:53 744402,-0.514465,-0.485779,0.287659,-0.645508,1.326660,...,-3,-1,1,0,0,False,0,0.398451,-1.233521,1.567341
2,2291479052128,warm,True,left,2019-02-14 14:46:53 752382,-0.514465,-0.485779,0.287659,-0.645508,1.326660,...,0,-1,-2,1,-1,False,0,0.398451,-1.233521,1.567341
3,2291479052128,warm,True,left,2019-02-14 14:46:53 752382,-0.514465,-0.485779,0.287659,-0.645508,1.326660,...,0,0,-1,-2,-1,False,0,0.398451,-1.233521,1.567341
4,2291479052128,warm,True,left,2019-02-14 14:46:53 756350,-0.519470,-0.484497,0.291138,-0.640930,1.186035,...,0,2,-1,1,1,False,0,0.420894,-1.246889,1.542884
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1555959,2291479052128,warm,True,left,2019-02-14 16:13:28 749540,-0.416260,0.036682,-0.058777,-0.906616,0.765625,...,7,-110,-117,-33,-12,False,0,0.117068,-0.861911,-3.114586
1555962,2291479052128,warm,True,left,2019-02-14 16:13:28 758516,-0.413208,0.032410,-0.063416,-0.907898,0.768066,...,-3,-118,-114,-28,-2,False,0,0.135021,-0.854758,-3.131782
1555963,2291479052128,warm,True,left,2019-02-14 16:13:28 758516,-0.413208,0.032410,-0.063416,-0.907898,0.768066,...,-1,-82,-128,-16,-2,False,0,0.135021,-0.854758,-3.131782
1555966,2291479052128,warm,True,left,2019-02-14 16:13:28 764500,-0.413208,0.032410,-0.063416,-0.907898,0.768066,...,12,-104,-119,-13,-1,False,0,0.135021,-0.854758,-3.131782


## Plot EMG and IMU Channels

In [4]:
''' Plot Entire EMG signals'''
for channel in range(1,9):
    plt.figure()
    ax = myo_df['EMG_' + str(channel)].plot()
    plt.title('EMG_' + str(channel))
    plt.ylabel('mVolts')
    plt.xlabel('Time')

for channel in ['X', 'Y', 'Z']:
    plt.figure()
    myo_df['Acc_' + channel].plot()
    plt.title('Acc_' + str(channel))
    plt.ylabel('g')
    plt.xlabel('Time')

## Get Descripitive Statistics for Raw Data

In [5]:
myo_df.describe()

,DeviceID,Orientation_W,Orientation_X,Orientation_Y,Orientation_Z,Acc_X,Acc_Y,Acc_Z,Gyro_X,Gyro_Y,...,EMG_3,EMG_4,EMG_5,EMG_6,EMG_7,EMG_8,RSSI,Roll,Pitch,Yaw
count,1.405884e+06,1.405884e+06,1.405884e+06,1.405884e+06,1.405884e+06,1.405884e+06,1.405884e+06,1.405884e+06,1.405884e+06,1.405884e+06,...,1.405884e+06,1.405884e+06,1.405884e+06,1.405884e+06,1.405884e+06,1.405884e+06,1405884.0,1.405884e+06,1.405884e+06,1.405884e+06
mean,2.291479e+12,5.829820e-02,-1.416711e-01,4.756671e-02,7.029249e-02,7.194029e-01,2.172123e-02,4.590001e-01,4.715409e-01,2.128901e-02,...,-8.544162e-01,-7.970885e-01,-8.382363e-01,-1.035961e+00,-7.810957e-01,-7.580533e-01,0.0,9.882653e-02,-9.217510e-01,6.220871e-03
std,1.109392e+02,3.596622e-01,6.225795e-01,3.632931e-01,5.660227e-01,3.237783e-01,3.134963e-01,2.499457e-01,3.317542e+01,2.453375e+01,...,1.196934e+01,7.937094e+00,1.137876e+01,1.678093e+01,9.360004e+00,1.066679e+01,0.0,4.923939e-01,4.655549e-01,1.682120e+00
min,2.291479e+12,-7.495117e-01,-9.933472e-01,-8.003540e-01,-9.848633e-01,-1.368164e+00,-2.659668e+00,-1.334473e+00,-4.955625e+02,-5.199375e+02,...,-1.280000e+02,-1.280000e+02,-1.280000e+02,-1.280000e+02,-1.280000e+02,-1.280000e+02,0.0,-3.140510e+00,-1.570796e+00,-3.141558e+00
25%,2.291479e+12,-2.727051e-01,-7.298584e-01,-2.425537e-01,-4.491577e-01,5.043945e-01,-1.855469e-01,2.695312e-01,-6.375000e+00,-3.437500e+00,...,-3.000000e+00,-3.000000e+00,-3.000000e+00,-4.000000e+00,-3.000000e+00,-3.000000e+00,0.0,-2.744367e-01,-1.277161e+00,-1.384925e+00
50%,2.291479e+12,1.106567e-01,-2.828979e-01,9.576416e-02,9.210205e-02,9.023438e-01,4.101562e-02,3.520508e-01,1.875000e-01,0.000000e+00,...,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,0.0,1.528443e-01,-1.194552e+00,1.360732e-01
75%,2.291479e+12,3.458252e-01,4.904785e-01,3.248901e-01,6.166992e-01,9.409180e-01,1.313477e-01,7.158203e-01,6.750000e+00,3.437500e+00,...,1.000000e+00,2.000000e+00,2.000000e+00,2.000000e+00,2.000000e+00,1.000000e+00,0.0,4.110263e-01,-5.247361e-01,1.339452e+00
max,2.291479e+12,7.894287e-01,9.920654e-01,7.677612e-01,9.845581e-01,2.842773e+00,1.588867e+00,1.886230e+00,5.243125e+02,4.203125e+02,...,1.270000e+02,1.270000e+02,1.270000e+02,1.270000e+02,1.270000e+02,1.270000e+02,0.0,3.139911e+00,1.480952e+00,3.141591e+00


## Rectify the Signal

In [6]:
rectified_df =myo_df.copy() # make copy of DF

for col in ['EMG_' + str(i) for i in range(1, 9)]:
    rectified_df[col] = rectified_df[col].apply(abs) # applys the absolute function to each channel
    
display(rectified_df)

,DeviceID,Warm?,Sync,Arm,Timestamp,Orientation_W,Orientation_X,Orientation_Y,Orientation_Z,Acc_X,...,EMG_4,EMG_5,EMG_6,EMG_7,EMG_8,Locked,RSSI,Roll,Pitch,Yaw
0,2291479052128,warm,True,left,2019-02-14 14:46:53 744402,-0.514465,-0.485779,0.287659,-0.645508,1.326660,...,2,1,1,1,1,False,0,0.398451,-1.233521,1.567341
1,2291479052128,warm,True,left,2019-02-14 14:46:53 744402,-0.514465,-0.485779,0.287659,-0.645508,1.326660,...,3,1,1,0,0,False,0,0.398451,-1.233521,1.567341
2,2291479052128,warm,True,left,2019-02-14 14:46:53 752382,-0.514465,-0.485779,0.287659,-0.645508,1.326660,...,0,1,2,1,1,False,0,0.398451,-1.233521,1.567341
3,2291479052128,warm,True,left,2019-02-14 14:46:53 752382,-0.514465,-0.485779,0.287659,-0.645508,1.326660,...,0,0,1,2,1,False,0,0.398451,-1.233521,1.567341
4,2291479052128,warm,True,left,2019-02-14 14:46:53 756350,-0.519470,-0.484497,0.291138,-0.640930,1.186035,...,0,2,1,1,1,False,0,0.420894,-1.246889,1.542884
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1555959,2291479052128,warm,True,left,2019-02-14 16:13:28 749540,-0.416260,0.036682,-0.058777,-0.906616,0.765625,...,7,110,117,33,12,False,0,0.117068,-0.861911,-3.114586
1555962,2291479052128,warm,True,left,2019-02-14 16:13:28 758516,-0.413208,0.032410,-0.063416,-0.907898,0.768066,...,3,118,114,28,2,False,0,0.135021,-0.854758,-3.131782
1555963,2291479052128,warm,True,left,2019-02-14 16:13:28 758516,-0.413208,0.032410,-0.063416,-0.907898,0.768066,...,1,82,128,16,2,False,0,0.135021,-0.854758,-3.131782
1555966,2291479052128,warm,True,left,2019-02-14 16:13:28 764500,-0.413208,0.032410,-0.063416,-0.907898,0.768066,...,12,104,119,13,1,False,0,0.135021,-0.854758,-3.131782


### Plot the Rectified Signals and Look at the new Means/Stdevs for each EMG Channel

In [7]:
'''Your Code Here'''
plt.figure(figsize=(15, 10))

for i in range(1, 9):
    col = 'EMG_' + str(i)
    plt.plot(rectified_df[col][:1000], label=col)  # First 1000 samples for visibility

plt.title("Rectified EMG Signals (First 1000 Samples)")
plt.xlabel("Sample")
plt.ylabel("Amplitude (Abs EMG)")
plt.legend()
plt.grid(True)
plt.show()
'''Stop Coding Here'''

'Stop Coding Here'

### Apply a rolling average with a window size of 400 (2 seconds) and look at the new plots
Look into Pandas rolling method (e.g, myo_df.rolling(400).mean())

In [8]:
'''Your code here'''
rolling_df = rectified_df.copy()

for i in range(1, 9):
    col = 'EMG_' + str(i)
    rolling_df[col] = rectified_df[col].rolling(window=400).mean()

# Plot the smoothed signals (first 2000 samples)
import matplotlib.pyplot as plt

plt.figure(figsize=(15, 10))
for i in range(1, 9):
    col = 'EMG_' + str(i)
    plt.plot(rolling_df[col][:2000], label=col)

plt.title("Smoothed EMG Signals with 400-sample Rolling Average")
plt.xlabel("Sample")
plt.ylabel("Amplitude")
plt.legend()
plt.grid(True)
plt.show()
'''Stop coding here'''

'Stop coding here'

### Advanced EMG Filtering
Now, we are going to apply a bandpass filter to each EMG channel.

In [9]:
import scipy as sp
import scipy.signal

def filteremg(emg, low_pass=3, sfreq=200, high_band=20, low_band=95):
    """
    emg: EMG data
    high: high-pass cut off frequency
    low: low-pass cut off frequency
    sfreq: sampling frequency
    """
    # Zero mean emg signal
    emg = emg - emg.mean()
    
    # normalise cut-off frequencies to sampling frequency
    high_band = high_band/(sfreq/2)
    low_band = low_band/(sfreq/2)
    
    
    # create bandpass filter for EMG
    b1, a1 = sp.signal.butter(4, [high_band,low_band], btype='bandpass', analog=True)
    
    # process EMG signal: filter EMG
    emg_filtered = sp.signal.filtfilt(b1, a1, emg)    
    
    # process EMG signal: rectify
    emg_rectified = abs(emg_filtered)
    
    # create lowpass filter and apply to rectified signal to get EMG envelope
    low_pass = low_pass/(sfreq/2)
    b2, a2 = sp.signal.butter(4, low_pass, fs=sfreq, btype='lowpass')
    emg_envelope = sp.signal.lfilter(b2, a2, emg_rectified)
    
    return emg_envelope
    

filt_emg = myo_df.copy()
emg_keys = ['EMG_' + str(i) for i in range(1, 9)]
filt_emg[emg_keys] = filt_emg[emg_keys].apply(filteremg, raw=True)
display(filt_emg)

,DeviceID,Warm?,Sync,Arm,Timestamp,Orientation_W,Orientation_X,Orientation_Y,Orientation_Z,Acc_X,...,EMG_4,EMG_5,EMG_6,EMG_7,EMG_8,Locked,RSSI,Roll,Pitch,Yaw
0,2291479052128,warm,True,left,2019-02-14 14:46:53 744402,-0.514465,-0.485779,0.287659,-0.645508,1.326660,...,2.272861e-14,3.934620e-15,2.257128e-13,4.746089e-14,1.901335e-14,False,0,0.398451,-1.233521,1.567341
1,2291479052128,warm,True,left,2019-02-14 14:46:53 744402,-0.514465,-0.485779,0.287659,-0.645508,1.326660,...,4.286240e-13,8.250328e-14,1.864408e-12,5.246762e-13,3.782821e-13,False,0,0.398451,-1.233521,1.567341
2,2291479052128,warm,True,left,2019-02-14 14:46:53 752382,-0.514465,-0.485779,0.287659,-0.645508,1.326660,...,2.811540e-12,5.700397e-13,8.042179e-12,2.751430e-12,2.543262e-12,False,0,0.398451,-1.233521,1.567341
3,2291479052128,warm,True,left,2019-02-14 14:46:53 752382,-0.514465,-0.485779,0.287659,-0.645508,1.326660,...,1.093093e-11,2.309576e-12,2.478039e-11,9.587392e-12,1.007211e-11,False,0,0.398451,-1.233521,1.567341
4,2291479052128,warm,True,left,2019-02-14 14:46:53 756350,-0.519470,-0.484497,0.291138,-0.640930,1.186035,...,3.096836e-11,6.825007e-12,6.179450e-11,2.596220e-11,2.902281e-11,False,0,0.420894,-1.246889,1.542884
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1555959,2291479052128,warm,True,left,2019-02-14 16:13:28 749540,-0.416260,0.036682,-0.058777,-0.906616,0.765625,...,2.817893e+01,3.844822e+01,4.296860e+01,2.827803e+01,3.349184e+01,False,0,0.117068,-0.861911,-3.114586
1555962,2291479052128,warm,True,left,2019-02-14 16:13:28 758516,-0.413208,0.032410,-0.063416,-0.907898,0.768066,...,2.817668e+01,3.845264e+01,4.297719e+01,2.827670e+01,3.349216e+01,False,0,0.135021,-0.854758,-3.131782
1555963,2291479052128,warm,True,left,2019-02-14 16:13:28 758516,-0.413208,0.032410,-0.063416,-0.907898,0.768066,...,2.817443e+01,3.845707e+01,4.298578e+01,2.827537e+01,3.349249e+01,False,0,0.135021,-0.854758,-3.131782
1555966,2291479052128,warm,True,left,2019-02-14 16:13:28 764500,-0.413208,0.032410,-0.063416,-0.907898,0.768066,...,2.817219e+01,3.846151e+01,4.299438e+01,2.827404e+01,3.349281e+01,False,0,0.135021,-0.854758,-3.131782


## Plot Filtered Signals and Get Mean/Stdev

In [10]:
''' Your code here'''

# Plot the filtered EMG signals (first 2000 samples for clarity)
plt.figure(figsize=(15, 10))
for i in range(1, 9):
    col = 'EMG_' + str(i)
    plt.plot(rolling_df[col][:2000], label=col)

plt.title("Filtered (Smoothed) EMG Signals with Rolling Average")
plt.xlabel("Sample")
plt.ylabel("Smoothed EMG Amplitude")
plt.legend()
plt.grid(True)
plt.show()

# Compute mean and standard deviation of the filtered signals
filtered_stats = {}
for i in range(1, 9):
    col = 'EMG_' + str(i)
    filtered_stats[col] = {
        'Mean': rolling_df[col].mean(),
        'Std Dev': rolling_df[col].std()
    }

# Display as a DataFrame
filtered_stats_df = pd.DataFrame(filtered_stats).T
display(filtered_stats_df)

''' Stop coding here'''

,Mean,Std Dev
EMG_1,4.671274,2.799111
EMG_2,5.458053,3.379120
EMG_3,5.960719,3.415275
EMG_4,4.607533,2.368714
EMG_5,6.245650,3.774770
EMG_6,8.805586,5.346261
EMG_7,5.321925,3.022640
EMG_8,5.411960,3.540980


' Stop coding here'

## Power Spectral Density
Now, we are going to look at a PSF plot

In [11]:
f, Pxx_den = sp.signal.periodogram(filt_emg['EMG_1'], 200)
plt.semilogy(f, Pxx_den)
plt.ylim([1e-7, 1e2])
plt.xlabel('frequency [Hz]')
plt.ylabel('PSD [V**2/Hz]')
plt.show()

### Plot the PSD for each filtered EMG channel and Find the Max Power

In [12]:
''' Your Code Here'''
from scipy import signal

# Dictionary to store max power per EMG channel
psd_max_power = {}

# Plot PSD for each EMG channel (filtered)
plt.figure(figsize=(15, 10))

for i in range(1, 9):
    col = 'EMG_' + str(i)
    
    # Compute PSD using scipy.signal.periodogram
    f, Pxx_den = signal.periodogram(rolling_df[col].dropna(), fs=200)  # 200 Hz sampling rate
    
    # Store the maximum power value
    psd_max_power[col] = Pxx_den.max()
    
    # Plot in semilog scale
    plt.semilogy(f, Pxx_den, label=col)

plt.title("Power Spectral Density of Filtered EMG Channels")
plt.xlabel("Frequency [Hz]")
plt.ylabel("PSD [V^2/Hz]")
plt.ylim([1e-7, 1e1])
plt.legend()
plt.grid(True)
plt.show()

# Convert max power dict to DataFrame
import pandas as pd
psd_max_df = pd.DataFrame.from_dict(psd_max_power, orient='index', columns=['Max PSD Power'])
display(psd_max_df)
''' Stop Coding Here'''

,Max PSD Power
EMG_1,3073.159756
EMG_2,5191.203512
EMG_3,6657.768935
EMG_4,2306.861064
EMG_5,4154.166559
EMG_6,10110.031800
EMG_7,5210.054319
EMG_8,3364.571493


' Stop Coding Here'

# Segmenting Data
Using your collected timestamps, we are now going to look at each gesture: rock, paper, scissors

In [13]:
'''First, we need to set pandas indexes to timestamps'''
myo_df.index = pd.to_datetime(myo_df['Timestamp'], format='%Y-%m-%d %H:%M:%S %f' )
display(myo_df)

,DeviceID,Warm?,Sync,Arm,Timestamp,Orientation_W,Orientation_X,Orientation_Y,Orientation_Z,Acc_X,...,EMG_4,EMG_5,EMG_6,EMG_7,EMG_8,Locked,RSSI,Roll,Pitch,Yaw
Timestamp,,,,,,,,,,,,,,,,,,,,,
2019-02-14 14:46:53.744402,2291479052128,warm,True,left,2019-02-14 14:46:53 744402,-0.514465,-0.485779,0.287659,-0.645508,1.326660,...,-2,1,-1,1,-1,False,0,0.398451,-1.233521,1.567341
2019-02-14 14:46:53.744402,2291479052128,warm,True,left,2019-02-14 14:46:53 744402,-0.514465,-0.485779,0.287659,-0.645508,1.326660,...,-3,-1,1,0,0,False,0,0.398451,-1.233521,1.567341
2019-02-14 14:46:53.752382,2291479052128,warm,True,left,2019-02-14 14:46:53 752382,-0.514465,-0.485779,0.287659,-0.645508,1.326660,...,0,-1,-2,1,-1,False,0,0.398451,-1.233521,1.567341
2019-02-14 14:46:53.752382,2291479052128,warm,True,left,2019-02-14 14:46:53 752382,-0.514465,-0.485779,0.287659,-0.645508,1.326660,...,0,0,-1,-2,-1,False,0,0.398451,-1.233521,1.567341
2019-02-14 14:46:53.756350,2291479052128,warm,True,left,2019-02-14 14:46:53 756350,-0.519470,-0.484497,0.291138,-0.640930,1.186035,...,0,2,-1,1,1,False,0,0.420894,-1.246889,1.542884
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-02-14 16:13:28.749540,2291479052128,warm,True,left,2019-02-14 16:13:28 749540,-0.416260,0.036682,-0.058777,-0.906616,0.765625,...,7,-110,-117,-33,-12,False,0,0.117068,-0.861911,-3.114586
2019-02-14 16:13:28.758516,2291479052128,warm,True,left,2019-02-14 16:13:28 758516,-0.413208,0.032410,-0.063416,-0.907898,0.768066,...,-3,-118,-114,-28,-2,False,0,0.135021,-0.854758,-3.131782
2019-02-14 16:13:28.758516,2291479052128,warm,True,left,2019-02-14 16:13:28 758516,-0.413208,0.032410,-0.063416,-0.907898,0.768066,...,-1,-82,-128,-16,-2,False,0,0.135021,-0.854758,-3.131782


In [14]:
'''Then we can segment data using the indexes'''
start = pd.to_datetime('2019-02-14 14:46:53')
stop  = pd.to_datetime('2019-02-14 14:48:53')
gesture_df = myo_df[start:stop]
display(gesture_df)
''' Notice that we did not need to specify the nano seconds'''

,DeviceID,Warm?,Sync,Arm,Timestamp,Orientation_W,Orientation_X,Orientation_Y,Orientation_Z,Acc_X,...,EMG_4,EMG_5,EMG_6,EMG_7,EMG_8,Locked,RSSI,Roll,Pitch,Yaw
Timestamp,,,,,,,,,,,,,,,,,,,,,
2019-02-14 14:46:53.744402,2291479052128,warm,True,left,2019-02-14 14:46:53 744402,-0.514465,-0.485779,0.287659,-0.645508,1.326660,...,-2,1,-1,1,-1,False,0,0.398451,-1.233521,1.567341
2019-02-14 14:46:53.744402,2291479052128,warm,True,left,2019-02-14 14:46:53 744402,-0.514465,-0.485779,0.287659,-0.645508,1.326660,...,-3,-1,1,0,0,False,0,0.398451,-1.233521,1.567341
2019-02-14 14:46:53.752382,2291479052128,warm,True,left,2019-02-14 14:46:53 752382,-0.514465,-0.485779,0.287659,-0.645508,1.326660,...,0,-1,-2,1,-1,False,0,0.398451,-1.233521,1.567341
2019-02-14 14:46:53.752382,2291479052128,warm,True,left,2019-02-14 14:46:53 752382,-0.514465,-0.485779,0.287659,-0.645508,1.326660,...,0,0,-1,-2,-1,False,0,0.398451,-1.233521,1.567341
2019-02-14 14:46:53.756350,2291479052128,warm,True,left,2019-02-14 14:46:53 756350,-0.519470,-0.484497,0.291138,-0.640930,1.186035,...,0,2,-1,1,1,False,0,0.420894,-1.246889,1.542884
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-02-14 14:47:27.604011,2291479052128,warm,True,left,2019-02-14 14:47:27 604011,0.150879,-0.792053,-0.540100,-0.241394,-1.056641,...,-4,36,-4,3,25,False,0,0.058538,1.190134,0.631412
2019-02-14 14:47:27.619038,2291479052128,warm,True,left,2019-02-14 14:47:27 619038,0.151001,-0.792786,-0.540466,-0.238037,-1.041992,...,-1,10,1,23,39,False,0,0.048260,1.191257,0.616196
2019-02-14 14:47:27.619038,2291479052128,warm,True,left,2019-02-14 14:47:27 619038,0.151001,-0.792786,-0.540466,-0.238037,-1.041992,...,3,-24,0,17,42,False,0,0.048260,1.191257,0.616196


' Notice that we did not need to specify the nano seconds'

## Reading in XDF
Now we are going to read in from an XDF file

In [15]:
''' Imports and useful functions'''
import pyxdf

def xdf_to_dataframe(xdf_data):
    ''' Xdf Data should be a list of streams (dictionaries)
        Function returns a dictionary of dataframes, one dataframe per stream'''  
    dataframes = {}
    for stream in xdf_data:
        df = pd.DataFrame()
        data = stream['time_series']
        timestamps = stream['time_stamps']
        df['Time'] = timestamps
        chan_names, units = get_channel_names(stream['info'])
        counts = data.shape[0]
        for series, name, unit in zip(range(data.shape[1]), chan_names, units):
            df[name[0]]  = data[:, series]
            if unit:
                df[name[0] + '_Unit'] = np.repeat(unit, counts)
        
        for item in stream['info']:
            if item not in ['name', 'desc', 'data']:
                try:
                    df[item] = np.repeat(stream['info'][item], counts)
                except:
                    continue
        dataframes[stream['info']['name'][0]] = df
        
    return dataframes
            
        
        

def get_channel_names(info):
    channels = info['desc'][0]['channels'][0]['channel']
    names = [chan['label'] for chan in channels ]
    units = [chan['unit'] for chan in channels ]
    return names, units

ModuleNotFoundError: No module named 'pyxdf'

In [ ]:
data, header = pyxdf.load_xdf('Data/test.xdf')
dfs = xdf_to_dataframe(data)
display(dfs['BioRadio-20312'])
''' Save new dataframe'''
dfs['BioRadio-20312'].to_csv('Data/test.csv')

# Your turn!
The above code introduced you to some simple EMG processing from already collected data, but now lets use your data.

For the Myo and BioRadio, do the following for each EMG channel:
- Import the files
- Segment the data for one Rock, Paper, and Scissors gesture based on your collected timestamps
- Filter each gesture using a bandpass filter and a notch filter (at 60 hz)
- Plot the Filtered EMG signals and the Power Spectral Density of each Gesture
- Determine the Max Power for each Gesture
- Determine the mean/standard deviation for each Gesture

Your report should include each plot in an appendix.

In the main body of your report, include a table for each gesture that has columns: Device, EMG Channel Number, Mean, Std Dev, Max Power. Fill out the rows